In [3]:
from keras import Input, layers

# tensor
input_tensor = Input(shape=(32,))

# layer object is function
dense = layers.Dense(32, activation='relu')

# layer object called by tensor, retuns tensor
output_tensor = dense(input_tensor)

In [6]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# the same old Sequential model
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Keras Functional API correspondent to layer model
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# Model class convert input/output tensor into model
model = Model(input_tensor, output_tensor)

# confirm model's architecture
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [8]:
unrelated_input = Input(shape=(32,))
bad_model = Model(unrelated_input, output_tensor)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_4:0", shape=(?, 64), dtype=float32) at layer "input_4". The following previous layers were accessed without issue: []

In [9]:
# compile model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# generate dummy train data
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

# train model through 10 epochs
model.fit(x_train, y_train, epochs=10, batch_size=128)

# evaluate model
score = model.evaluate(x_train, y_train)
print(score)


Epoch 1/10
1000/1000 [==============================] - 0s 469us/step - loss: 11.8753
Epoch 2/10
1000/1000 [==============================] - 0s 21us/step - loss: 12.6930
Epoch 3/10
1000/1000 [==============================] - 0s 23us/step - loss: 14.3356
Epoch 4/10
1000/1000 [==============================] - 0s 22us/step - loss: 16.7798
Epoch 5/10
1000/1000 [==============================] - 0s 20us/step - loss: 19.5315
Epoch 6/10
1000/1000 [==============================] - 0s 29us/step - loss: 22.5610
Epoch 7/10
1000/1000 [==============================] - 0s 20us/step - loss: 25.6270
Epoch 8/10
1000/1000 [==============================] - 0s 21us/step - loss: 29.2558
Epoch 9/10
1000/1000 [==============================] - 0s 18us/step - loss: 33.2598
Epoch 10/10
1000/1000 [==============================] - 0s 52us/step
40.64219006347656


In [13]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# text input is variable length sequence
# and if needed, we can name input
text_input = Input(shape=(None,), dtype='int32', name='text')

# embed input into sequence of size 64 vector
""" 2020.02.16 change
embedded_text = layers.Embedding(
    text_vocabulary_size)(text_input, 64)
"""
embedded_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input)

# encode this into single vector through LSTM
encoded_text = layers.LSTM(32)(embedded_text)

# also repeat the same process for question input
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# concatenate eoncoded text and question
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# add softmax classifier
answer = layers.Dense(
    answer_vocabulary_size, activation='softmax')(concatenated)

# specify 2 inputs and 1 output on instantiation of model
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [14]:
import numpy as np
num_samples = 1000
max_length = 100

# generate dummy NumPy data
text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                            size=(num_samples, max_length))

# apply one-hot encoding for answer (not integer value)
answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
for i, x in enumerate(answers):
    x[indices[i]] = 1
    
# adopt input list
model.fit([text, question], answers, epochs=10, batch_size=128)


Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2148 - acc: 0.0030
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1988 - acc: 0.0570
Epoch 3/10
1000/1000 [==============================] - 1s 757us/step - loss: 6.1555 - acc: 0.0160
Epoch 4/10
1000/1000 [==============================] - 1s 885us/step - loss: 6.0740 - acc: 0.0050
Epoch 5/10
1000/1000 [==============================] - 1s 945us/step - loss: 6.0017 - acc: 0.0100
Epoch 6/10
1000/1000 [==============================] - 1s 762us/step - loss: 5.9301 - acc: 0.0160
Epoch 7/10
1000/1000 [==============================] - 1s 933us/step - loss: 5.8438 - acc: 0.0210
Epoch 8/10
1000/1000 [==============================] - 1s 811us/step - loss: 5.7541 - acc: 0.0260
Epoch 9/10
1000/1000 [==============================] - 1s 828us/step - loss: 5.6669 - acc: 0.0370
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.5995 - acc: 0.0450


In [15]:
# adopt input dictionary (only case of having name)
model.fit({'text': text, 'question': question}, answers,
         epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 1s 789us/step - loss: 5.5393 - acc: 0.0520
Epoch 2/10
1000/1000 [==============================] - 1s 866us/step - loss: 5.4591 - acc: 0.0500
Epoch 3/10
1000/1000 [==============================] - 1s 905us/step - loss: 5.4085 - acc: 0.0590
Epoch 4/10
1000/1000 [==============================] - 1s 807us/step - loss: 5.3324 - acc: 0.0650
Epoch 5/10
1000/1000 [==============================] - 1s 774us/step - loss: 5.2701 - acc: 0.0780
Epoch 6/10
1000/1000 [==============================] - 1s 848us/step - loss: 5.2371 - acc: 0.0780
Epoch 7/10
1000/1000 [==============================] - 1s 784us/step - loss: 5.1420 - acc: 0.0940
Epoch 8/10
1000/1000 [==============================] - 1s 789us/step - loss: 5.1089 - acc: 0.0970
Epoch 9/10
1000/1000 [==============================] - 1s 805us/step - loss: 5.0328 - acc: 0.1140
Epoch 10/10
1000/1000 [==============================] - 1s 855us/step - loss: 4.9786 - acc: 0.1320


In [19]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# remark that output layers have name
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                activation='softmax',
                                name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input,
             [age_prediction, income_prediction, gender_prediction])

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, None, 128)    163968      embedding_9[0][0]                
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, None, 128)    0           conv1d_16[0][0]                  
____________________________________________________________________________________________

In [23]:
model.compile(optimizer='rmsprop',
             loss=['mse',
                 'categorical_crossentropy',
                 'binary_crossentropy'])

# same as above
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                  'income': 'categorical_crossentropy',
                  'gender': 'binary_crossentropy'})

In [24]:
model.compile(optimizer='rmsprop',
             loss=['mse',
                 'categorical_crossentropy',
                 'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])

# same as above
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                  'income': 'categorical_crossentropy',
                  'gender': 'binary_crossentropy'},
             loss_wights={'age': 0.25, 'income': 1., 'gender': 10.})